<a href="https://colab.research.google.com/github/WHU-Peter/COMP6248-Deep-Learning/blob/master/Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer
import torch
from torchvision import transforms 
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchbearer import Trial
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch import nn
from torch import optim

In [ ]:
class MyDataset(Dataset):
  def __init__(self , size=5000, dim=40, random_offset=0):
    super(MyDataset , self ).__init__() 
    self.size = size
    self.dim = dim
    self.random_offset = random_offset

  def __getitem__( self , index ): 
    if index>=len(self):
      raise IndexError('{} index out of range'.format(self.__class__.__name__))
    rng_state = torch.get_rng_state()
    torch.manual_seed(index + self .random_offset)
    while True :
      img = torch.zeros(self.dim, self.dim)
      dx = torch.randint(-10,10,(1,),dtype=torch.float) 
      dy = torch.randint(-10,10,(1,),dtype=torch.float) 
      c = torch.randint(-20,20,(1,) , dtype=torch.float)

      params = torch.cat((dy/dx, c))
      xy = torch.randint(0,img.shape[1] , (20, 2), dtype=torch.float)
      xy[: ,1] = xy[: ,0] * params[0] + params[1]
      xy.round_()
      xy = xy[ xy[:,1] > 0 ]
      xy = xy[ xy[:,1] < self.dim ] 
      xy = xy[ xy[:,0] < self.dim ]
      for i in range(xy.shape[0]):
        x, y = xy[i][0], self.dim - xy[i][1] 
        img[int(y), int(x)]=1
      if img.sum() > 2: 
        break
    torch.set_rng_state(rng_state) 
    return img.unsqueeze(0), params
  
  def __len__(self):
    return self.size

train_data = MyDataset()
val_data = MyDataset(size=500, random_offset=33333) 
test_data = MyDataset(size=500, random_offset=99999)

# create data loaders
trainloader = DataLoader(train_data, batch_size=128, shuffle=True)
valloader = DataLoader(val_data, batch_size=128, shuffle=True)
testloader = DataLoader(test_data, batch_size=128, shuffle=True)

In [ ]:
class RegressionCNN(nn.Module):
    def __init__(self):
        super(RegressionCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 48, (3, 3), padding=1)
        self.fc1 = nn.Linear(48 * 40**2, 128)
        self.fc2 = nn.Linear(128, 2)
            
    def forward(self, x):
        out = self.conv1(x)
        out = F.relu(out)
        out = out.view(out.shape[0], -1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        return out

In [ ]:
# build the model
model = RegressionCNN()

# define the loss function and the optimiser
loss_function = nn.MSELoss()
optimiser = optim.Adam(model.parameters(), weight_decay=0.1)

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(trainloader, val_generator=valloader, test_generator=testloader)
trial.run(epochs=200)
results = trial.evaluate(data_key=torchbearer.TEST_DATA)
print(results)

NameError: ignored

In [ ]:
class RegressionCNN2(nn.Module):
    def __init__(self):
        super(RegressionCNN2, self).__init__()
        self.conv1 = nn.Conv2d(1, 48, (3, 3), padding=1)
        self.fc1 = nn.Linear(48 * 40 * 1, 128)
        self.fc2 = nn.Linear(128, 2)
            
    def forward(self, x):
        out = self.conv1(x)
        out = F.relu(out)
        out = self.conv1(x)
        out = F.relu(out)
        out = nn.AdaptiveMaxPool2d((40, 1))(out)
        out = out.view(out.shape[0], -1)
        # out = F.dropout(out, 0.3)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        return out

In [ ]:
# build the model
model = RegressionCNN2()

# define the loss function and the optimiser
loss_function = nn.MSELoss()
optimiser = optim.Adam(model.parameters(), weight_decay=0.1)

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(trainloader, val_generator=valloader, test_generator=testloader)
trial.run(epochs=200)
results = trial.evaluate(data_key=torchbearer.TEST_DATA)
print(results)


{'test_loss': 14.540283203125, 'test_mse': 14.502610206604004}
